In [1]:
%run ../yp_utils.py

# Initial setup

In [2]:
paper_pmid = 27440388
paper_name = 'zhao_liu_2016' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['dataset_id', 'name'])

In [4]:
datasets.set_index('dataset_id', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/Beidong Percentage of cells with synphilin-1 aggregates-SGA-0803.xlsx', sheet_name='Sheet1', skiprows=3)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5376 x 5


In [7]:
original_data.head()

,ORF,GENE,COMMENT,P- value T-TEST n=2,Dif % of mother with aggregates
0,YER115C,SPR6,0,1.50277e-05,-59.8027
1,YER129W,PAK1,0,1.50277e-05,-59.8027
2,YGL024W,YGL024W,0,0.00028304,-55.5246
3,YER155C,BEM2,0,0.000244836,-54.5226
4,YDR245W,MNN10,0,0.000400303,-52.9241


In [8]:
original_data['orf'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['orf'] = clean_orf(original_data['orf'])

In [10]:
# Translate to ORFs 
original_data['orf'] = translate_sc(original_data['orf'], to='orf')

In [11]:
# Make sure everything translated ok
t = looks_like_orf(original_data['orf'])
print(original_data.loc[~t,])

Empty DataFrame
Columns: [ORF, GENE, COMMENT, P- value T-TEST n=2, Dif % of mother with aggregates, orf]
Index: []


In [12]:
original_data['data'] = pd.to_numeric(original_data['Dif % of mother with aggregates'], errors='coerce')

In [13]:
original_data.set_index('orf', inplace=True)

In [14]:
original_data = original_data[['data']].copy()

In [15]:
original_data = original_data.groupby(original_data.index).mean()

In [16]:
original_data.shape

(4268, 1)

# Prepare the final dataset

In [17]:
data = original_data[['data']].copy()

In [18]:
dataset_ids = [4825]
datasets = datasets.reindex(index=dataset_ids)

In [19]:
lst = [datasets.index.values, ['value']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data.columns = idx

In [20]:
data.head()

dataset_id,4825
data_type,value
orf,
YAL002W,-10.143772
YAL004W,-7.869224
YAL005C,4.156098
YAL007C,-3.219517
YAL008W,-2.279358


## Subset to the genes currently in SGD

In [21]:
genes = pd.read_csv(path_to_genes, sep='\t', index_col='id')
genes = genes.reset_index().set_index('systematic_name')
gene_ids = genes.reindex(index=data.index.values)['id'].values
num_missing = np.sum(np.isnan(gene_ids))
print('ORFs missing from SGD: %d' % num_missing)

ORFs missing from SGD: 22


In [22]:
data['gene_id'] = gene_ids
data = data.loc[data['gene_id'].notnull()]
data['gene_id'] = data['gene_id'].astype(int)
data = data.reset_index().set_index(['gene_id','orf'])

In [23]:
data.head()

,dataset_id,4825
,data_type,value
gene_id,orf,
2,YAL002W,-10.143772
1863,YAL004W,-7.869224
4,YAL005C,4.156098
5,YAL007C,-3.219517
6,YAL008W,-2.279358


# Normalize

In [24]:
data_norm = normalize_phenotypic_scores(data, has_tested=True)

In [25]:
# Assign proper column names
lst = [datasets.index.values, ['valuez']*datasets.shape[0]]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','data_type'])
data_norm.columns = idx

In [26]:
data_norm[data.isnull()] = np.nan
data_all = data.join(data_norm)

In [27]:
data_all.head()


dataset_id            4825          
data_type            value    valuez
gene_id orf                         
2       YAL002W -10.143772 -1.220055
1863    YAL004W  -7.869224 -0.989603
4       YAL005C   4.156098  0.228772
5       YAL007C  -3.219517 -0.518507
6       YAL008W  -2.279358 -0.423252

# Print out

In [28]:
for f in ['value','valuez']:
    df = data_all.xs(f, level='data_type', axis=1).copy()
    df.columns = datasets['name'].values
    df = df.droplevel('gene_id', axis=0)
    df.to_csv(paper_name + '_' + f + '.txt', sep='\t')

# Save to DB

In [29]:
from IO.save_data_to_db3 import *

In [30]:
save_data_to_db(data_all, paper_pmid)

  0%|          | 0/1 [00:00<?, ?it/s]

Deleting all datasets for PMID 27440388...
Inserting the new data...


100%|██████████| 1/1 [00:06<00:00,  6.90s/it]

Updating the data_modified_on field...
